Run workflow_coarse_reg -> workflow_elastic_reg to get here

Copied from notebook, need to tweak data/variables for out data.

In [1]:
from sofima.processor import warp   # Appears like I need to import tensorflow
import numpy as np
import tensorstore as ts

from connectomics.common import bounding_box
from connectomics.common import box_generator
from connectomics.volume import subvolume

2023-06-02 23:12:22.272081: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-02 23:12:22.329824: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Prep renderer to read data
from fine_registration import SyncAdapter
import zarr_io

GCS_BUCKET = 'sofima-test-bucket'

class StitchAndRender3dTiles(warp.StitchAndRender3dTiles):
  cache = {}

  # Edited this to collapse 5D zarr -> 3D tiles.
  def _open_tile_volume(self, tile_id: int):
    if tile_id in self.cache:
      return self.cache[tile_id]

    vol = zarr_io.open_zarr(GCS_BUCKET, f"tile_X_000{tile_id}_Y_0000_Z_0000_CH_0405_cam0.zarr/2")
    vol = vol[0,0,:,:,:] # convert to zyx axis layout (diff from coarse and fine)

    self.cache[tile_id] = SyncAdapter(vol, tile_id)
    return self.cache[tile_id]

In [3]:
# Prep renderer inputs

# +---> x
# |
# v y
tile_layout = np.array([[1], 
                        [2],
                        [3],
                        [4],
                        [5],
                        [6]])

Genuine question-- how to find boundaries of image dimensions?
Communication thread with Michal-- pretty much need to run into error -> request help. 


In [4]:
# Prep output buffer
# This outputs n5, but not possible to output 5D volumes 

# GCS_BUCKET = "sofima-test-bucket"
# DATASET_PATH = "stitched_st20"
# ds_out = ts.open({
#   'driver': 'zarr',
#   'kvstore': {
#       'driver': 'gcs',
#       'bucket': GCS_BUCKET,
#   },
#   'create': True,
#   'delete_existing': True,
#   'path': DATASET_PATH,
#   'metadata': {
#     'compression': {'type': 'blosc', 'cname': 'lz4', 'clevel': 5, 'shuffle': 1},
#     'dataType': 'uint8',
#     'dimensions': [1590, 9740, 7650],
#     'blockSize': [128, 128, 128],
#     'axes': ['z', 'y', 'x'],
#     'resolution': [13,10,10],
#     'units': ['nm', 'nm', 'nm']
#   },
# }).result()

import zarr_io

bucket = "sofima-test-bucket"
shape = [1, 1, 3544, 576 * 3.5, 576]   # 3.5 comes from overlap calculation
path = 'fused_6.zarr'
ds_out = zarr_io.write_zarr(bucket, shape, path)

I0000 00:00:1685747552.818379   26902 gcs_resource.cc:102] Using default AdmissionQueue with limit 32
I0000 00:00:1685747552.820769   27035 google_auth_provider.cc:179] Running on GCE, using service account 895865026362-compute@developer.gserviceaccount.com


In [5]:
# Initialize the renderer

# Tile pattern path will not fly for other naming conventions, 
# but works fine for dispim

tile_pattern_path = "tile_X_000{tile_id}_Y_0000_Z_0000_CH_0405_cam0.zarr/2"
tile_mesh_path = "solved_mesh_st20_6.npz"

renderer = StitchAndRender3dTiles(
    tile_map=tile_layout,
    tile_pattern_path=tile_pattern_path,
    tile_mesh_path=tile_mesh_path,
    stride=(20, 20, 20),
    offset=[0, 0, 0],  # Trying the y offset
    parallelism=8
)

### NOTE:

Must load in the elastic data structure everytime fusion is run. 
Fusion will set all these values by zero. 

In [6]:
# Call renderer.process on subvolumes solved meshes

# NOTE: 
# Need to adapt the output from 3D -> 5D on write to ds_out. 
# I sense an identical fusion algorithm here: 
# - Define output domain chunks
# - Determine input images within output domain chunks
# - Sum/Combine these input images
# - Repeat for each output chunk

box = bounding_box.BoundingBox(start=(0,0,0), size=ds_out.shape[4:1:-1])  # Needs xyz 
gen = box_generator.BoxGenerator(box, (512, 512, 512), (0, 0, 0), True) # These are xyz
renderer.set_effective_subvol_and_overlap((512, 512, 512), (0, 0, 0))
# TODO: Tweak output chunk size

# box_info = []

import time
for i, sub_box in enumerate(gen.boxes):
    t_start = time.time()

    # Feed in an empty subvol, with dimensions of sub_box. 
    inp_subvol = subvolume.Subvolume(np.zeros(sub_box.size[::-1], dtype=np.uint16)[None, ...], sub_box)
    ret_subvol = renderer.process(inp_subvol)  # czyx
    # box_info.append(np.all(ret_subvol==0))
    # print(f'Ret subvol: {np.all(ret_subvol==0)}')

    t_render = time.time()

    # ret_subvol is a 4D CZYX volume
    slice = ret_subvol.bbox.to_slice3d()
    slice = (0, 0, slice[0], slice[1], slice[2])
    ds_out[slice].write(ret_subvol.data[0, ...]).result()
    
    t_write = time.time()
    
    print('box {i}: {t1:0.2f} render  {t2:0.2f} write'.format(i=i, t1=t_render - t_start, t2=t_write - t_render))

box 0: 0.63 render  0.05 write
box 1: 0.23 render  0.12 write
(slice(0, 541, None), slice(0, 326, None), slice(0, 541, None))
(slice(0, 542, None), slice(0, 41, None), slice(0, 541, None))
box 2: 37.43 render  0.42 write
(slice(0, 541, None), slice(0, 326, None), slice(17, 576, None))
(slice(0, 542, None), slice(0, 41, None), slice(16, 576, None))
box 3: 16.63 render  0.59 write
(slice(0, 541, None), slice(241, 566, None), slice(0, 541, None))
(slice(0, 542, None), slice(0, 540, None), slice(0, 541, None))
(slice(0, 542, None), slice(0, 274, None), slice(0, 539, None))
box 4: 54.50 render  0.48 write
(slice(0, 541, None), slice(241, 566, None), slice(15, 576, None))
(slice(0, 542, None), slice(0, 540, None), slice(14, 576, None))
(slice(0, 542, None), slice(0, 274, None), slice(14, 576, None))
box 5: 46.14 render  0.69 write
(slice(0, 541, None), slice(436, 576, None), slice(0, 538, None))
(slice(0, 541, None), slice(151, 575, None), slice(0, 538, None))
(slice(0, 542, None), slice(0, 

In [ ]:
# 27 minutes run for original chunk size
# 40 minutes run for smaller chunk size, also a blank output. 
# |-- to get smaller chunks, perform re-chunking, as directed by Camilo. 

In [17]:
# Deeper analysis of boxes:
box = bounding_box.BoundingBox(start=(0,0,0), size=ds_out.shape[4:1:-1])  # Needs xyz 
gen = box_generator.BoxGenerator(box, (512, 512, 512), (0, 0, 0), True)
renderer.set_effective_subvol_and_overlap((512, 512, 512), (0, 0, 0))

# help(gen)
# DIG INTO renderer.process and offset next. 

# And first 3 boxes:
ret_subvols = []
for i, sub_box in enumerate(list(gen.boxes)[:4]):
    inp_subvol = subvolume.Subvolume(np.zeros(sub_box.size[::-1], dtype=np.uint8)[None, ...], sub_box)
    ret_subvol = renderer.process(inp_subvol)
    ret_subvols.append(ret_subvol)

    # ret_subvol is a 4D CZYX volume
    slice = ret_subvol.bbox.to_slice3d()
    slice = (0, 0, slice[0], slice[1], slice[2])
    ds_out[slice].write(ret_subvol.data[0, ...]).result()

(slice(0, 460, None), slice(0, 186, None), slice(199, 581, None))
(slice(0, 460, None), slice(0, 186, None), slice(258, 581, None))
(slice(0, 460, None), slice(103, 566, None), slice(195, 581, None))
(slice(0, 461, None), slice(0, 420, None), slice(194, 580, None))
(slice(0, 461, None), slice(0, 134, None), slice(194, 578, None))
(slice(0, 460, None), slice(103, 566, None), slice(256, 581, None))
(slice(0, 461, None), slice(0, 420, None), slice(254, 580, None))
(slice(0, 460, None), slice(0, 134, None), slice(254, 578, None))
